In [43]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [44]:
def printest(args, value):
    return print( "{} : \n {} \n".format(args, value) )

# Initial Data Preparation

Churn prediction is about identifying customers who are likely to cancel their contracts soon. If the company can do that, it can offer discounts on these services in an effort to keep the users. Here we use the dataset of churn prediction for a telecom company.

In [45]:
df = pd.read_csv('Data/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head(2)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No


In [46]:
df.head(1).T

,0
customerID,7590-VHVEG
gender,Female
SeniorCitizen,0
Partner,Yes
Dependents,No
tenure,1
PhoneService,No
MultipleLines,No phone service
InternetService,DSL
OnlineSecurity,No


We see that the dataset has a few columns:
- CustomerID: the ID of the customer
- Gender: male/female
- SeniorCitizen: whether the customer is a senior citizen (0/1)
- Partner: whether they live with a partner (yes/no)
- Dependents: whether they have dependents (yes/no)
- Tenure: number of months since the start of the contract
- PhoneService: whether they have phone service (yes/no)
- MultipleLines: whether they have multiple phone lines (yes/no/no phone service)
- InternetService: the type of internet service (no/fiber/optic)
- OnlineSecurity: if online security is enabled (yes/no/no internet)
- OnlineBackup: if online backup service is enabled (yes/no/no internet)
- DeviceProtection: if the device protection service is enabled (yes/no/no internet)
- TechSupport: if the customer has tech support (yes/no/no internet)
- StreamingTV: if the TV streaming service is enabled (yes/no/no internet)
- StreamingMovies: if the movie streaming service is enabled (yes/no/no internet)
- Contract: the type of contract (monthly/yearly/two years)
- PaperlessBilling: if the billing is paperless (yes/no)
- PaymentMethod: payment method (electronic check, mailed check, bank transfer,
credit card)
- MonthlyCharges: the amount charged monthly (numeric)
- TotalCharges: the total amount charged (numeric)
- Churn: if the client has canceled the contract (yes/no)

When import a CSV file, Pandas tries to guess the right type for each column. But sometimes, it doesn't get it right. So, it's a good idea to double-check the types using ``df.dtypes``.

In [47]:
df.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

We observe that the 'TotalCharges' column poses an issue. Rather than being classified as a numeric type, such as float or integer, pandas incorrectly infers it as an object type.    

In [48]:
# Convert 'TotalCharges' to numeric, replace non-numeric with NaN
df['TotalCharges'] = pd.to_numeric(df.TotalCharges, errors='coerce')

# Create a filter for NaN values
filter = df['TotalCharges'].isna()

# Display the rows where 'TotalCharges' was NaN
print('Before:')
display(df[filter][['customerID','TotalCharges']].head(2))

# Fill NaN values with zero
df['TotalCharges'] = df['TotalCharges'].fillna(0)

# Display the rows where 'TotalCharges' was NaN before the fillna operation
print('After:')
display(df[filter][['customerID','TotalCharges']].head(2))

Before:


,customerID,TotalCharges
488,4472-LVYGI,NaN
753,3115-CZMZD,NaN


After:


,customerID,TotalCharges
488,4472-LVYGI,0.0
753,3115-CZMZD,0.0


In [49]:
# Columns

# lowering columns name and replace spaces by _
df_columns_lower = df.columns.str.lower()
df.columns  = df_columns_lower.str.replace(' ', '_')

# Rows

# boolean mask for columns with strings
column_mask = df.dtypes == 'object' 
string_columns = list(df.dtypes[column_mask].index)

# lowering rows strings and replace spaces by _
for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

df.head(2)

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-vhveg,female,0,yes,no,1,no,no_phone_service,dsl,no,...,no,no,no,no,month-to-month,yes,electronic_check,29.85,29.85,no
1,5575-gnvde,male,0,no,no,34,yes,no,dsl,yes,...,yes,no,no,no,one_year,no,mailed_check,56.95,1889.50,no


We the that some columns has 'yes' or 'no' string, that we can convert to boolean. First consider the target variable churn

In [50]:
df.churn = (df.churn == 'yes').astype(int)

df[['customerid', 'churn']].head().T

,0,1,2,3,4
customerid,7590-vhveg,5575-gnvde,3668-qpybk,7795-cfocw,9237-hqitu
churn,0,0,1,0,1


In [51]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state = 1)
df_train, df_val= train_test_split(df_train_full, test_size=0.33, random_state = 11)

y_train = df_train['churn'].values
y_val = df_val['churn'].values

del df_train['churn']
del df_val['churn']

display(df_train.head(2))

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges
2935,9435-jmlsx,male,0,yes,no,71,yes,no,dsl,yes,yes,yes,yes,yes,yes,two_year,yes,bank_transfer_(automatic),86.1,6045.9
3639,0512-flfdw,female,1,yes,no,60,yes,yes,fiber_optic,no,no,yes,no,yes,yes,one_year,yes,credit_card_(automatic),100.5,6029.0


# Exploratory Data Analysis (EDA)

We have already found a problem with the TotalCharges column and replaced the missing values with zeros. Now let’s see if we need to perform any additional null handling:

In [52]:
df_train_full.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

Let’s check the proportion of churned users among all customers. This is the **Global Churn Rate** that refers to the overall churn rate for the entire customer base of the dataset.

For that, we need to divide the number of customers who churned by the total number of customers as follows:

In [77]:
# checking the distribution of values in the target variable
churn_stats = df_train_full['churn'].agg([pd.value_counts])

# Mean
total_values = churn_stats['value_counts'].sum()
churn_stats['global_mean'] = round(churn_stats['value_counts']/total_values, 3)

display(churn_stats)


,value_counts,global_mean
0,4113,0.73
1,1521,0.27


This gives us the proportion of churned users, or the probability that a customer will churn. As we see in, approximately 27% of the customers stopped
using our services, and the rest remained as customers. 

Also, the dataset is a imbalanced one. There were three times as many people who didn’t churn in our dataset as those who did churn.

Let's separate the dataset in categorical and numerical variables:

In [64]:
# All categorical columns except 'customerid'
categorical_mask = df_train.dtypes == 'object'
categorical = list(df_train.dtypes[categorical_mask].index)
categorical.remove('customerid') 

# Manually add 'seniorcitizen' because it's an int boolean (0 or 1)
categorical.append('seniorcitizen')
printest('categorical', categorical)

# All numerical columns except 'seniorcitizen' because it's an int boolean
numerical_mask = df_train.dtypes != 'object'
numerical = list(df_train.dtypes[numerical_mask].index)
numerical.remove('seniorcitizen')
printest('numerical', numerical)

categorical : 
 ['gender', 'partner', 'dependents', 'phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod', 'seniorcitizen'] 

numerical : 
 ['tenure', 'monthlycharges', 'totalcharges'] 



In [65]:
#Count number of distinct elements in specified axis.
df_train_full[categorical].nunique()

gender              2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
seniorcitizen       2
dtype: int64

## Feature importance

**Global Churn Ratio**

In [78]:
display(churn_stats)

,value_counts,global_mean
0,4113,0.73
1,1521,0.27


**Group Churn Ratio**

The churn rate within a specific customer segment, known as the group churn rate, allows for targeted analysis. By comparing this group churn rate with the overall churn rate, we can better understand how a particular group's behavior deviates from the average customer behavior. If there's a minimal difference between the group and global churn rates, it indicates that this specific group's churn behavior is not significantly different from the overall customer base. Therefore, this group's characteristics might not be a critical factor in predicting churn.

Now, let's start our analysis with the gender variable:

In [74]:
gender_mean = df_train_full.groupby('gender')['churn'].mean()
display(gender_mean)

gender
female    0.276824
male      0.263214
Name: churn, dtype: float64

The difference between the group rates for both males and females is quite small, which indicates that knowing the gender of the customer
doesn’t help us identify whether they will churn.

Now let’s take a look at another variable: partner:

In [75]:
partner_mean = df_train_full.groupby('partner')['churn'].mean()
display(partner_mean)

partner
no     0.329809
yes    0.205033
Name: churn, dtype: float64

The churn rate for people with a partner is significantly less than the rate for the ones without a partner — 20.5% versus 33%. It means that clients with no partner are more likely to churn than the ones with a partner

**Risk Ratio**

In addition to looking at the difference between the churn group rate and the churn global rate,
it’s interesting to look at the ratio between them. In statistics, the ratio between probabilities
in different groups is called the risk ratio. Here, 'risk' denotes the likelihood of experiencing the event under investigation. In our case, it's the risk of churning.

$$\text{Risk} =\frac{\text{Group Rate}}{\text{Global Rate}} $$

the risk can assume values between zero and infinity. It has a nice interpretation that tells you
how likely the elements of the group are to have the effect (churn) compared with the
entire population.

- $\text{Risk} > 1$: More churn in the group than in the population
- $\text{Risk} = 1$: same level of risk as the rest of the population
- $\text{Risk} < 1$: group has lower risk
  

